In [78]:
from typing import List, Tuple
from random import choices, randint, randrange, random
from collections import namedtuple
from functools import partial
Genome = List[int]
Population = List[Genome]
Thing = namedtuple('Thing', ['name', 'value', 'weight'])

things = [
    Thing('Laptop', 500, 2200),
    Thing('headphones',150, 160),
    Thing('Coffee Mug', 60, 350), 
    Thing('Notepad', 40, 333), 
    Thing('Water Bottle', 30, 192)
]

def generate_genome(length: int) -> Genome:
    return choices([0,1], k=length) #generate random list of 1's and 0's of the appropriate length (k)
def generate_population(size:int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]
def fitness(genome: Genome, things: [Thing], weight_limit: int ) -> int:
    if len(genome) != len(things):
        raise ValueError('Genome and things must be of the same length')
    weight = 0
    value = 0 
    for i, thing in enumerate(things):
        if genome[i] == 1:
            weight += thing.weight
            value += thing.value

            if weight > weight_limit: #if weight is greater than the limit, it should return 0
                return 0 
    return value

def selection_pair(population: Population, fitness_func) -> Population:
    return choices(
        population=population,
        weights=[fitness_func(genome) for genome in population],
        k=2 #tells the function to draw twice, getting a pair
    )

def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError('a and b need to be equal length')
    length = len(a)
    if length < 2:
        return a,b
    p = randint(1, length - 1 )
    return a[0:p] + b[p:], b[0:p] + a[p:]

#flips bits if random num less than probability 
def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome

In [79]:
def run_evolution(populate_func,fitness_func, fitness_limit: int, selection_func = selection_pair, crossover_func = single_point_crossover, mutation_func = mutation, generation_limit: int = 100) -> Tuple[Population,int]:
    population = populate_func()

    for i in range(generation_limit):
        population = sorted(
            population,
            key=lambda genome: fitness_func(genome),
            reverse=True
        )        
        if fitness_func(population[0]) >= fitness_limit:
            break
        #puts the best 2 results in the next generation
        next_generation = population[0:2]
        #for half of the length of the current population
        for j in range(int(len(population) / 2 ) - 1):
            #selection_func picks a pair from the current population, weighted by how good the solution is
            parents = selection_func(population, fitness_func)
            #crosses the genome of the parents
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            #mutates children
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            #adds two children to the next gen. This is why the length is half of the current population
            next_generation += [offspring_a, offspring_b]

        population = next_generation
    population = sorted(
        population, 
        key= lambda genome: fitness_func(genome),
        reverse = True
    )
    return population, i, fitness_func(population[0])

population, generations, value = run_evolution(
    populate_func = partial(generate_population, size=10,genome_length = len(things)),
    fitness_func = partial(fitness, things=things, weight_limit=3000),
    fitness_limit= 740,
    generation_limit=100
)

def genome_to_things(genome: Genome, things: [Thing]) -> [Thing]:
    result = []
    for i, thing in enumerate(things):
        if genome[i] == 1:
            result += [thing.name]

    return result

print(f"Number of generations: {generations}")
print(f"Best Solution: {genome_to_things(population[0], things)}")
print(f"Best solution score: {value}")


Number of generations: 36
Best Solution: ['Laptop', 'headphones', 'Coffee Mug', 'Water Bottle']
Best solution score: 740
